In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在[Vertex AI](https://cloud.google.com/vertex-ai)上训练、调整和部署PyTorch文本分类模型
细调预训练的[BERT](https://huggingface.co/bert-base-cased)模型以进行情感分类任务

# 概述

这个示例源自Token-Classification [notebook](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb)和run_glue.py (https://github.com/huggingface/transformers/blob/v2.5.0/examples/run_glue.py)。
我们将对**`bert-base-cased`**（预训练）模型进行微调，用于情感分类任务。
您可以在[Hugging Face Hub](https://huggingface.co/bert-base-cased)找到有关该模型的详细信息。

要了解关于最先进的PyTorch/Tensorflow/JAX的更多notebooks，您可以探索[Hugging FaceNotebooks](https://huggingface.co/transformers/notebooks.html)。

### 数据集

我们将使用[Hugging Face Datasets](https://huggingface.co/datasets)中的[IMDB电影评论数据集](https://huggingface.co/datasets/imdb)。

### 目标

如何在[Vertex AI](https://cloud.google.com/vertex-ai)上构建、训练、调整和部署PyTorch模型，并强调对在Vertex AI上训练和部署PyTorch模型的一流支持。

### 目录

这个notebook涵盖了以下部分：

- [创建Notebooks实例](#Creating-Notebooks-instance-on-Google-Cloud)
- [训练](#Training)
    - [在Notebook中本地运行训练](#Training-locally-in-the-notebook)
    - [在Vertex AI上运行训练作业](#Training-on-Vertex-AI)
        - [使用预构建容器进行训练](#Run-Custom-Job-on-Vertex-AI-Training-with-a-pre-built-container)
        - [使用自定义容器进行训练](#Run-Custom-Job-on-Vertex-AI-Training-with-custom-container)
- [调整](#Hyperparameter-Tuning) 
    - [在Vertex AI上运行超参数调整作业](#Run-Hyperparameter-Tuning-Job-on-Vertex-AI)
- [部署](#Deploying)
    - [在Vertex AI Predictions上使用自定义容器部署模型](#Deploying-model-on-Vertex AI-Predictions-with-custom-container)

### 成本

本教程使用Google Cloud Platform (GCP)的计费组件：

* [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)
* [Vertex AI Training](https://cloud.google.com/vertex-ai/docs/training/custom-training)
* [Vertex AI Predictions](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)
* [Cloud Storage](https://cloud.google.com/storage)
* [Container Registry](https://cloud.google.com/container-registry)
* [Cloud Build](https://cloud.google.com/build) *[可选]*

了解[Vertex AI Pricing](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage Pricing](https://cloud.google.com/storage/pricing)和[Cloud Build Pricing](https://cloud.google.com/build/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

在谷歌云上创建笔记本实例

此笔记本假设你正在使用带有GPU运行时的PyTorch 1.9 DLVM开发环境。您可以使用谷歌云控制台或`gcloud`命令创建一个笔记本实例。

```
gcloud notebooks instances create example-instance \
    --vm-image-project=deeplearning-platform-release \
    --vm-image-family=pytorch-1-9-cu110-notebooks \
    --machine-type=n1-standard-4 \
    --location=us-central1-a \
    --boot-disk-size=100 \
    --accelerator-core-count=1 \
    --accelerator-type=NVIDIA_TESLA_V100 \
    --install-gpu-driver \
    --network=default
```
***
**注意：**在创建带有GPU的实例之前，您必须先确认您有GPU配额。请检查[配额](https://console.cloud.google.com/iam-admin/quotas)页面以确保您的项目中有足够的GPU可用。如果GPU未在配额页面列出或您需要额外的GPU配额，请[请求提高配额](https://cloud.google.com/compute/quotas#requesting_additional_quota)。免费试用账户默认不会获得GPU配额。

### 设置您的本地开发环境

**如果您正在使用Colab或Google Cloud笔记本**，您的环境已经符合运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合本笔记本的要求。您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一个简化的指南：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)
2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)
3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。
4. 要安装Jupyter，在终端中的命令行中运行`pip3 install jupyter`。
5. 要启动Jupyter，在终端中的命令行中运行`jupyter notebook`。
6. 在Jupyter Notebook仪表板中打开这个笔记本。

### 安装额外的包

此笔记本所需的Python依赖项包括[Transformers](https://pypi.org/project/transformers/)，[Datasets](https://pypi.org/project/datasets/)和[hypertune](https://github.com/GoogleCloudPlatform/cloudml-hypertune)将在笔记本实例内部安装。

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
!pip -q install {USER_FLAG} --upgrade transformers
!pip -q install {USER_FLAG} --upgrade datasets
!pip -q install {USER_FLAG} --upgrade tqdm
!pip -q install {USER_FLAG} --upgrade cloudml-hypertune

我们将使用[Python的Vertex AI SDK](https://cloud.google.com/vertex-ai/docs/start/client-libraries#python)与Vertex AI服务进行交互。高级的`aiplatform`库旨在通过使用包装器类和特定的默认值来简化常见的数据科学工作流程。

#### 安装Python的Vertex AI SDK

In [ ]:
!pip -q install {USER_FLAG} --upgrade google-cloud-aiplatform

重新启动内核

在安装额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在开始之前

选择一个GPU运行时

如果有这个选项，请确保你在GPU运行时中运行这个笔记本。在Colab中，选择“运行时 --> 更改运行时类型 > GPU”。

### 设置您的 Google Cloud 项目

**无论您使用的是哪种笔记本环境，都需要按照以下步骤操作。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您会获得$300 的免费信用额度用于支付计算/存储成本。
1. [确保您的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。
1. 在您的项目中启用以下 API，这些 API 是运行教程所必需的
    - [Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
    - [Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=storage.googleapis.com)
    - [Container Registry API](https://console.cloud.google.com/flows/enableapi?apiid=containerregistry.googleapis.com)
    - [Cloud Build API](https://console.cloud.google.com/flows/enableapi?apiid=cloudbuild.googleapis.com)
1. 如果您正在本地运行这个笔记本，您将需要安装 [Cloud SDK](https://cloud.google.com/sdk)。
1. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 会将以 `!` 开头的行视为 shell 命令，它会将以 `$` 开头的 Python 变量插值到这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`或`google.auth`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # <---CHANGE THIS TO YOUR PROJECT

import os

# Get your Google Cloud project ID using google.auth
if not os.getenv("IS_TESTING"):
    import google.auth

    _, PROJECT_ID = google.auth.default()
    print("Project ID: ", PROJECT_ID)

# validate PROJECT_ID
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    print(
        f"Please set your project id before proceeding to next step. Currently it's set as {PROJECT_ID}"
    )

时间戳

如果您正在参加直播教程，您可能会使用共享测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到您在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime


def get_timestamp():
    return datetime.now().strftime("%Y%m%d%H%M%S")


TIMESTAMP = get_timestamp()
print(f"TIMESTAMP = {TIMESTAMP}")

### 验证您的Google Cloud账户

---

**如果您正在使用Google Cloud笔记本**，您的环境已经通过验证。请跳过此步骤。

如果您正在使用Colab，请运行下面的单元格并按照提示进行身份验证以通过oAuth进行身份验证。

否则，请按照以下步骤操作：

1. 在Cloud控制台中，转到[创建服务帐号密钥页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。
2. 点击**创建服务帐号**。
3. 在**服务帐号名称**字段中输入名称，然后点击**创建**。
4. 在**授予此服务帐号对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中输入“Vertex AI”，并选择**Vertex AI管理员**。在过滤框中输入“存储对象管理员”，然后选择**存储对象管理员**。
5. 点击*创建*。包含您的密钥的JSON文件将下载到本地环境中。
6. 在下面的单元格中输入您服务帐号密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量，并运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

使用 Cloud SDK 提交训练作业时，您需要将包含训练代码的 Python 包上传到一个云存储桶中。Vertex AI 将从该包中运行代码。在本教程中，Vertex AI 还将在同一个存储桶中保存训练作业产生的训练模型。使用这个模型工件，您可以创建 Vertex AI 模型和端点资源，以便提供在线预测。

请在下面设置您的云存储桶名称。它必须在所有的云存储桶中是唯一的。

您也可以更改 `REGION` 变量，它将在本笔记本的其余操作中使用。确保选择一个 Vertex AI 服务可用的区域。您不能使用多区域存储桶进行 Vertex AI 训练。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # <---CHANGE THIS TO YOUR BUCKET
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = f"gs://{PROJECT_ID}aip-{get_timestamp()}"

In [ ]:
print(f"PROJECT_ID = {PROJECT_ID}")
print(f"BUCKET_NAME = {BUCKET_NAME}")
print(f"REGION = {REGION}")

只有当您的存储桶尚不存在时: 运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 导入库并定义常量

In [ ]:
import base64
import json
import os
import random
import sys

import google.auth
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip
from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google.protobuf.json_format import MessageToDict

In [ ]:
from IPython.display import HTML, display

In [ ]:
import datasets
import numpy as np
import pandas as pd
import torch
import transformers
from datasets import ClassLabel, Sequence, load_dataset
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          EvalPrediction, Trainer, TrainingArguments,
                          default_data_collator)

In [ ]:
print(f"Notebook runtime: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print(f"PyTorch version : {torch.__version__}")
print(f"Transformers version : {datasets.__version__}")
print(f"Datasets version : {transformers.__version__}")

In [ ]:
APP_NAME = "finetuned-bert-classifier"

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 训练

在这一部分，我们将通过微调[Hugging Face Transformers](https://github.com/huggingface/transformers)的预训练模型来训练一个PyTorch模型。我们首先会在本地训练模型，然后在[Vertex AI训练服务](https://cloud.google.com/vertex-ai/docs/training/custom-training)上进行训练。

在笔记本上进行本地培训

加载数据集

对于这个例子，我们将使用[Hugging Face Datasets](https://huggingface.co/datasets/)中的[IMDB电影评论数据集](https://huggingface.co/datasets/imdb)来进行情感分类任务。我们使用[Hugging Face Datasets](https://github.com/huggingface/datasets)库来下载数据。可以通过`load_dataset`函数很容易地完成这个任务。

In [ ]:
dataset = load_dataset("imdb")
dataset

`dataset`对象本身是[`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict)，其中包含了训练集、验证集和测试集的一个键。

In [ ]:
print(
    "Total # of rows in training dataset {} and size {:5.2f} MB".format(
        dataset["train"].shape[0], dataset["train"].size_in_bytes / (1024 * 1024)
    )
)
print(
    "Total # of rows in test dataset {} and size {:5.2f} MB".format(
        dataset["test"].shape[0], dataset["test"].size_in_bytes / (1024 * 1024)
    )
)

要访问一个实际元素，您需要首先选择一个拆分，然后给出一个索引：

In [ ]:
dataset["train"][0]

使用`unique`方法提取标签列表。这将使我们能够在不进行硬编码标签的情况下尝试其他数据集。

In [ ]:
label_list = dataset["train"].unique("label")
label_list

为了了解数据的样貌，以下函数将在数据集中随机选择一些示例（自动解码标签）。

In [ ]:
def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

### 数据预处理

在我们可以将这些文本馈送给我们的模型之前，我们需要对它们进行预处理。这是通过一个预训练的Hugging Face Transformers [`Tokenizer`类](https://huggingface.co/transformers/main_classes/tokenizer.html)来完成的，该类会对输入进行tokenize（包括将token转换为预训练词汇中的相应ID），然后将其放入模型期望的格式，同时生成模型需要的其他输入。

为了做到这一切，我们使用`AutoTokenizer.from_pretrained`方法实例化我们的tokenizer，这样可以确保：

- 我们得到一个与我们想要使用的模型架构相对应的tokenizer，
- 我们下载用于预训练特定检查点时使用的词汇表。

该词汇表将被缓存，因此下次运行单元格时不会重新下载。

In [ ]:
batch_size = 16
max_seq_length = 128
model_name_or_path = "bert-base-cased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    use_fast=True,
)
# 'use_fast' ensure that we use fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

您可以通过快速分词器在[模型大表格](https://huggingface.co/transformers/index.html#bigtable)上检查可用模型的类型。

你可以直接在一个句子上调用这个分词器：

In [ ]:
tokenizer("Hello, this is one sentence!")

根据您选择的模型，您将在上面单元格返回的字典中看到不同的键。对于我们在这里要做的事情来说并不重要（只需知道它们是后面我们要实例化的模型所需的），如果您感兴趣，可以在[此教程](https://huggingface.co/transformers/preprocessing.html)中了解更多信息。

**注意：** 如果像这里一样，您的输入已经被分成单词，您应该使用参数`is_split_into_words=True`将单词列表传递给您的标记器。

In [ ]:
example = dataset["train"][4]
print(example)

In [ ]:
tokenizer(
    ["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],
    is_split_into_words=True,
)

请注意，transformers通常使用子词分词器进行预训练，这意味着即使您的输入已被分割为单词，这些单词中的每一个也可能被分割为更小的子词。让我们看一个例子：

In [ ]:
# Dataset loading repeated here to make this cell idempotent
# Since we are over-writing datasets variable
dataset = load_dataset("imdb")

# Mapping labels to ids
# NOTE: We can extract this automatically but the `Unique` method of the datasets
# is not reporting the label -1 which shows up in the pre-processing.
# Hence the additional -1 term in the dictionary
label_to_id = {1: 1, 0: 0, -1: 0}


def preprocess_function(examples):
    """
    Tokenize the input example texts
    NOTE: The same preprocessing step(s) will be applied
    at the time of inference as well.
    """
    args = (examples["text"],)
    result = tokenizer(
        *args, padding="max_length", max_length=max_seq_length, truncation=True
    )

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [label_to_id[example] for example in examples["label"]]

    return result


# apply preprocessing function to input examples
dataset = dataset.map(preprocess_function, batched=True, load_from_cache_file=True)

微调模型

现在我们的数据准备好了，我们可以下载预先训练好的模型并对其进行微调。

微调涉及采用已经为特定任务训练过的模型，然后调整该模型以用于另一个类似任务。具体而言，调整涉及复制预先训练好的模型中的所有层，包括权重和参数，但不包括输出层。然后添加一个新的输出分类器层，用于预测当前任务的标签。最后一步是从头开始训练输出层，而预先训练模型的所有层参数都被冻结。这样可以学习来自预先训练表示的内容，并对更适合具体任务的更高级特征表示进行“微调”，例如在这种情况下分析情感。

对于笔记本中分析情感的情景，预先训练的BERT模型已经对语言的许多信息进行了编码，因为该模型是以自监督方式在大型英语数据语料库上进行训练的。现在我们只需要稍微调整它们，使用它们的输出作为情感分类任务的特征。这意味着在一个更小的数据集上进行更快的开发迭代，而不是使用更大的训练数据集训练特定的自然语言处理（NLP）模型。

由于我们所有的任务都涉及令牌分类，我们使用`AutoModelForSequenceClassification`类。与分词器一样，`from_pretrained`方法将为我们下载并缓存模型。我们唯一需要指定的是我们问题的标签数量（我们可以从之前看到的特征中获取）。

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path, num_labels=len(label_list)
)

**注意：**警告告诉我们正在丢弃一些权重（`vocab_transform`和`vocab_layer_norm`层），同时随机初始化另一些权重（`pre_classifier`和`classifier`层）。在这种情况下，这是绝对正常的，因为我们正在移除用于在掩模语言建模目标上预训练模型的头部，并用一个我们没有预训练权重的新头部替换它，因此库警告我们在使用此模型进行推理之前应该微调此模型，这正是我们要做的。

要实例化一个`Trainer`，我们需要定义另外三个事物。最重要的是[`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments)，它是一个包含所有属性以自定义训练的类。它需要一个文件夹名称，用于保存模型的检查点，其他所有参数都是可选的。

In [ ]:
args = TrainingArguments(
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    output_dir="/tmp/cls",
)

在这里，我们将评估设置为在每个时代结束时进行，调整学习率，使用笔记本顶部定义的“batch_size”并自定义训练的时代数量，以及权重衰减。

为我们的`Trainer`定义的最后一件事是如何从预测中计算指标。您可以定义自定义的compute_metrics函数。它接受一个`EvalPrediction`对象（具有predictions和label_ids字段的namedtuple），并且必须返回一个字符串到浮点数的字典。

In [ ]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

现在我们创建“Trainer”对象，几乎可以准备好开始训练了。

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=default_data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

您可以向`trainer`对象添加回调函数，以自定义训练循环的行为，例如提前停止、在评估阶段结束时报告指标或做出任何决策。在此笔记本的超参数调整部分，我们为`trainer`添加了一个回调函数，以自动化超参数调整过程。

我们现在可以通过调用`train`方法来微调我们的模型：

In [ ]:
trainer.train()

In [ ]:
saved_model_local_path = "./models"
!mkdir ./models

In [ ]:
trainer.save_model(saved_model_local_path)

`evaluate` 方法允许您在评估数据集上再次评估或在另一个数据集上评估。

In [ ]:
history = trainer.evaluate()

In [ ]:
history

要获得计算的其他度量指标，如每个类别的精度、召回率或F1分数，我们可以在`predict`方法的结果上应用与之前相同的函数。

### 使用示例样本在本地运行预测

使用训练好的模型，我们可以在应用训练期间使用的预处理函数后，预测输入文本的情感标签。我们将在笔记本中本地运行预测，然后展示如何使用[TorchServe](https://pytorch.org/serve/)在Vertex AI Predictions上部署模型。

In [ ]:
model_name_or_path = "bert-base-cased"
label_text = {0: "Negative", 1: "Positive"}
saved_model_path = saved_model_local_path


def predict(input_text, saved_model_path):
    # initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

    # preprocess and encode input text
    tokenizer_args = (input_text,)
    predict_input = tokenizer(
        *tokenizer_args,
        padding="max_length",
        max_length=128,
        truncation=True,
        return_tensors="pt",
    )

    # load trained model
    loaded_model = AutoModelForSequenceClassification.from_pretrained(saved_model_path)

    # get predictions
    output = loaded_model(predict_input["input_ids"])

    # return labels
    label_id = torch.argmax(*output.to_tuple(), dim=1)

    print(f"Review text: {input_text}")
    print(f"Sentiment : {label_text[label_id.item()]}\n")

In [ ]:
# example #1
review_text = (
    """Jaw dropping visual affects and action! One of the best I have seen to date."""
)
predict_input = predict(review_text, saved_model_path)

In [ ]:
# example #2
review_text = """Take away the CGI and the A-list cast and you end up with film with less punch."""
predict_input = predict(review_text, saved_model_path)

## 在Vertex AI上的培训

您可以在Notebooks实例上进行本地实验。但是，对于更大的数据集或模型，通常需要垂直扩展计算或水平分布式训练。执行此任务的最有效方式是利用[Vertex AI定制训练服务](https://cloud.google.com/vertex-ai/docs/training/custom-training)，原因如下：

- **自动提供和去掉资源**：在Vertex AI上进行训练作业将自动提供计算资源，执行训练任务，并确保一旦训练作业完成就删除计算资源。
- **可重用性和可移植性**：您可以将训练代码及其参数和依赖项打包到一个容器中，并创建一个可移植的组件。然后可以在不同的场景下运行此容器，例如超参数调整、不同的数据源等。
- **规模训练**：您可以在AI上运行一个[分布式训练作业](https://cloud.google.com/vertex-ai/docs/training/distributed-training)，允许您在集群中跨多个节点并行训练模型，从而缩短训练时间。
- **日志记录和监控**：训练服务将作业的消息记录到[Cloud Logging](https://cloud.google.com/logging/docs)，可以在作业运行时进行监控。

在这部分笔记本中，我们展示如何通过打包代码并创建训练管道来扩展Vertex AI的训练作业。使用[Vertex AI定制训练服务](https://cloud.google.com/vertex-ai/docs/training/custom-training)运行训练作业有三个步骤：

- **步骤1**：确定训练代码结构 - 打包为Python源分发或自定义容器镜像
- **步骤2**：选择自定义训练方法 - 自定义作业、超参数训练作业或训练管道
- **步骤3**：运行训练作业

![custom-training-on-vertex-ai](./images/custom-training-on-vertex-ai.png)

#### 定制训练方法

您可以在Vertex AI上创建三种类型的资源来训练自定义模型：

- **[自定义作业](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)**：通过自定义作业，您可以配置在Vertex AI上运行训练代码的设置，如工作池规格 - 机器类型、加速器、Python训练规格或自定义容器规格。
- **[超参数调整作业](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning)**：超参数调整作业根据您配置的标准自动调整模型的超参数，例如优化的目标/指标、超参数值和运行的试验次数。
- **[训练管道](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline)**：在成功完成训练作业后，编排自定义训练作业或超参数调整作业的附加步骤。

请参考[文档](https://cloud.google.com/vertex-ai/docs/training/custom-training-methods)了解更多详细信息。

在本笔记本中，我们将介绍自定义作业和超参数调整作业。

### 打包训练应用程序

在 Vertex AI 上运行训练作业之前，训练应用程序的代码和任何依赖项必须被打包并上传到您的 Google Cloud 项目可以访问的 Cloud Storage 存储桶、容器注册表或 Artifact Registry 中。本节展示了如何在云中打包和部署您的应用程序。

有两种方式可以打包您的应用程序和依赖项并在 Vertex AI 上进行训练：

1.使用[Python源代码分发](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)，将训练代码和依赖项打包，并与 Vertex AI 上的[预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)一起使用
2.使用[Docker容器](https://cloud.google.com/ai-platform/training/docs/custom-containers-training)打包依赖项来使用[自定义容器](https://cloud.google.com/ai-platform/training/docs/custom-containers-training)

**此笔记本展示了在 Vertex AI 上运行自定义训练作业的两种打包选项。**

#### 推荐的培训应用程序结构

您可以根据喜好构建培训应用程序的结构。然而，[以下结构](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container#structure)在Vertex AI示例中常用，使您的项目组织结构类似于示例可以更轻松地跟随示例。

我们有两个目录 `python_package` 和 `custom_container` 显示了两种打包方法。每个目录中的 `README.md` 文件中详细说明了目录结构和如何在本地和云端运行应用程序的指导。

```
.
├── custom_container
│   ├── Dockerfile
│   ├── README.md
│   ├── scripts
│   │   └── train-cloud.sh
│   └── trainer -> ../python_package/trainer/
├── python_package
│   ├── README.md
│   ├── scripts
│   │   └── train-cloud.sh
│   ├── setup.py
│   └── trainer
│       ├── __init__.py
│       ├── experiment.py
│       ├── metadata.py
│       ├── model.py
│       ├── task.py
│       └── utils.py
└── pytorch-text-classification-vertex-ai-train-tune-deploy.ipynb    --> 这个笔记本
```

1. 主项目目录包含您的`setup.py`文件或带有依赖项的`Dockerfile`。
2. 使用名为`trainer`的子目录来存储您的主应用程序模块和`scripts`用于提交本地或云端的培训作业。
3. 在`trainer`目录内：
    - `task.py` - 主应用程序模块，初始化和解析任务参数(超参数)，以及训练的入口点。
    - `model.py` - 包括使用预训练模型创建带有序列分类头的模型的函数。
    - `experiment.py` - 运行模型训练和评估实验，并导出最终模型。
    - `metadata.py` - 定义用于分类任务的元数据，如预定义模型数据集名称、目标标签。
    - `utils.py` - 包括诸如读取数据的数据输入函数、将模型保存到GCS存储桶等实用函数。

在Vertex AI Training上使用预构建的容器运行自定义作业

Vertex AI提供Docker容器映像，可以作为[预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#available_container_images)用于自定义训练。这些容器包括在基于机器学习框架和框架版本的训练代码中常用的依赖项。

在这个笔记本中，我们正在使用Hugging Face Datasets，并使用PyTorch对Hugging Face Transformers库中的变换器模型进行微调，用于情感分析任务。我们将使用[PyTorch的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#pytorch)，并通过在`setup.py`文件中添加标准Python依赖项`transformers`、`datasets`和`tqdm`来打包训练应用程序代码。

![在Vertex AI训练中使用预构建容器进行训练](./images/training-with-prebuilt-containers-on-vertex-training.png)

将变量初始化为定义预先构建的容器映像、训练应用程序的位置和训练模块。

In [ ]:
PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-7:latest"
)

PYTHON_PACKAGE_APPLICATION_DIR = "python_package"

source_package_file_name = f"{PYTHON_PACKAGE_APPLICATION_DIR}/dist/trainer-0.1.tar.gz"
python_package_gcs_uri = (
    f"{BUCKET_NAME}/pytorch-on-gcp/{APP_NAME}/train/python_package/trainer-0.1.tar.gz"
)
python_module_name = "trainer.task"

以下是用于训练应用程序的`setup.py`文件。`setup.py`中的`find_packages()`函数包括`trainer`目录在包中，因为它包含了`__init__.py`，告诉[Python Setuptools](https://setuptools.readthedocs.io/en/latest/)将父目录的所有子目录包含为依赖项。

In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/setup.py

from setuptools import find_packages
from setuptools import setup
import setuptools

from distutils.command.build import build as _build
import subprocess


REQUIRED_PACKAGES = [
    'transformers',
    'datasets',
    'tqdm',
    'cloudml-hypertune'
]

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Vertex AI | Training | PyTorch | Text Classification | Python Package'
)

运行以下命令来创建一个源分发包，dist/trainer-0.1.tar.gz:

In [ ]:
!cd {PYTHON_PACKAGE_APPLICATION_DIR} && python3 setup.py sdist --formats=gztar

现在将包含训练应用程序的源分发上传到云存储桶

In [ ]:
!gsutil cp {source_package_file_name} {python_package_gcs_uri}

验证源分发存在于云存储桶中

In [ ]:
!gsutil ls -l {python_package_gcs_uri}

#### *[可选]* **在本地运行自定义训练任务**

在将工作提交到云端之前，您可以通过直接调用`trainer.task`模块在本地运行训练任务。

In [ ]:
!cd {PYTHON_PACKAGE_APPLICATION_DIR} && python -m trainer.task

在Vertex AI上运行自定义训练作业

我们使用[Python的Vertex AI SDK](https://cloud.google.com/vertex-ai/docs/start/client-libraries#client_libraries)来创建和提交训练作业到Vertex AI训练服务。

初始化Python版的Vertex AI SDK

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

配置和提交自定义作业到Vertex AI训练服务

使用[预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)镜像和封装为Python源代码分发的PyTorch训练代码，配置一个[自定义作业](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)。

**注意：**在使用Vertex AI SDK for Python提交训练作业时，它会创建一个[训练管道](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline)，该管道在Vertex AI训练服务上启动自定义作业。

In [ ]:
print(f"APP_NAME={APP_NAME}")
print(
    f"PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI={PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI}"
)
print(f"python_package_gcs_uri={python_package_gcs_uri}")
print(f"python_module_name={python_module_name}")

In [ ]:
JOB_NAME = f"{APP_NAME}-pytorch-pkg-ar-{get_timestamp()}"
print(f"JOB_NAME={JOB_NAME}")

In [ ]:
job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=f"{JOB_NAME}",
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI,
)

In [ ]:
training_args = ["--num-epochs", "2", "--model-name", "finetuned-bert-classifier"]

model = job.run(
    replica_count=1,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1,
    args=training_args,
    sync=False,
)

**监控自定义工作进度**

您可以通过以下链接[此处](https://console.cloud.google.com/vertex-ai/training/training-pipelines/)监控从云控制台启动的自定义工作，或者使用gcloud CLI命令[`gcloud beta ai custom-jobs stream-logs`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs)

![在Vertex AI培训中监控自定义工作进度](./images/vertex-training-monitor-custom-job.png)

在作业成功完成后，通过训练代码验证写入GCS的模型构件。

In [ ]:
job_response = MessageToDict(job._gca_resource._pb)
gcs_model_artifacts_uri = job_response["trainingTaskInputs"]["baseOutputDirectory"][
    "outputUriPrefix"
]
print(f"Model artifacts are available at {gcs_model_artifacts_uri}")

In [ ]:
!gsutil ls -lr $gcs_model_artifacts_uri/

### 使用 gcloud CLI 使用 Python 源代码分发提交自定义作业

您可以使用 [`gcloud beta ai custom-jobs create`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/create) 命令将训练作业提交到 Vertex AI 训练服务。`gcloud` 命令将您的训练应用程序暂存到 GCS 存储桶并提交训练作业。

```
gcloud beta ai custom-jobs create \
    --display-name=${JOB_NAME} \
    --region ${REGION} \
    --python-package-uris=${PACKAGE_PATH} \
    --worker-pool-spec=replica-count=1,machine-type='n1-standard-8',accelerator-type='NVIDIA_TESLA_V100',accelerator-count=1,executor-image-uri=${IMAGE_URI},python-module='trainer.task',local-package-path="../python_package/" \
    --args="--model-name","finetuned-bert-classifier","--job-dir",$JOB_DIR
```

- `worker-pool-spec` 参数定义了自定义作业使用的工作池配置。以下是 `worker-pool-spec` 中的字段：
    - 将`executor-image-uri`设置为`us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-7:latest`以便在预构建的 PyTorch v1.7 GPU 图像上进行训练
    - 将 `local-package-path` 设置为训练代码的路径
    - 将 `python-module` 设置为 `trainer.task`，该模块是启动应用程序的主要模块
    - 设置`accelerator-type`和`machine-type`以设置运行应用程序的计算类型

有关更多详细信息，请参阅[文档](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/create#--args)。

在 `./python_package/scripts/train-cloud.sh` 中的脚本包含了启动自定义作业和监视日志的 `gcloud` 命令。

In [ ]:
!cd python_package && ./scripts/train-cloud.sh

除了Cloud Console之外，您还可以通过传递作业ID使用[gcloud CLI](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs)来流式传输日志以监控作业进度：

```
gcloud ai custom-jobs stream-logs <job_id> --region=$REGION
```

您可以通过运行以下命令验证写入GCS的模型文件，以通过训练代码来运行模型：

```
!gsutil ls -l $JOB_DIR/
```

在Vertex AI训练中使用自定义容器运行自定义作业

要创建一个自定义容器的培训工作，您需要定义一个 `Dockerfile` 来安装或添加训练工作所需的依赖项。然后，您需要在本地构建和测试 Docker 镜像以进行验证，将镜像推送到容器注册表，并提交一个自定义作业到 Vertex AI 训练服务。

![在 Vertex AI 上使用自定义容器进行训练](./images/training-with-custom-containers-on-vertex-training.png)

使用Dockerfile和训练代码以及依赖项构建您的容器

在先前的部分中，我们将训练应用程序代码和依赖项包装为 Python 源分发包。打包训练应用程序和依赖项的另一种方法是使用 Dockerfile 创建自定义容器。我们创建一个 Dockerfile，使用由 Vertex AI 提供的预构建的 PyTorch 容器映像作为基本映像，安装依赖项 - `transformers`、`datasets`、`tqdm` 和 `cloudml-hypertune`，并复制训练应用程序代码。

In [ ]:
%%writefile ./custom_container/Dockerfile

# Use pytorch GPU base image
FROM us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-10:latest

# set working directory
WORKDIR /app

# Install required packages
RUN pip install google-cloud-storage transformers datasets tqdm cloudml-hypertune

# Copies the trainer code to the docker image.
COPY ./trainer/__init__.py /app/trainer/__init__.py
COPY ./trainer/experiment.py /app/trainer/experiment.py
COPY ./trainer/utils.py /app/trainer/utils.py
COPY ./trainer/metadata.py /app/trainer/metadata.py
COPY ./trainer/model.py /app/trainer/model.py
COPY ./trainer/task.py /app/trainer/task.py

# Set up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

构建镜像并标记容器注册表路径（gcr.io），以便推送。

In [ ]:
CUSTOM_TRAIN_IMAGE_URI = f"gcr.io/{PROJECT_ID}/pytorch_gpu_train_{APP_NAME}"

In [ ]:
!cd ./custom_container/ && docker build -f Dockerfile -t $CUSTOM_TRAIN_IMAGE_URI ../python_package

#### *[可选]* **使用自定义容器在本地运行训练任务**

以分离模式在本地运行容器进行测试。在带有GPU的机器上运行时，您可以使用`--gpus all`命令行标志。

In [ ]:
!docker run --gpus all -it --rm $CUSTOM_TRAIN_IMAGE_URI

在Vertex AI上运行自定义容器的训练作业

在将训练作业提交到Vertex AI之前，将自定义容器镜像推送到Google Cloud容器注册表，然后提交训练作业给Vertex AI。

注意：[容器注册表](https://cloud.google.com/container-registry)是一个中央仓库，用于存储、管理和保护您的Docker容器镜像。

将容器推送到容器注册表

将带有训练应用程序代码和依赖项的容器镜像推送到您的容器注册表。

In [ ]:
!docker push $CUSTOM_TRAIN_IMAGE_URI

验证容器注册表中的自定义容器镜像.

In [ ]:
!gcloud container images describe $CUSTOM_TRAIN_IMAGE_URI

初始化 Python 的 Vertex AI SDK

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

配置并提交自定义作业到 Vertex AI 训练服务

使用包含训练代码和其他依赖项的[自定义容器](https://cloud.google.com/vertex-ai/docs/training/create-custom-container)图像配置一个[自定义作业](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)

**注意:** 当使用 Vertex AI SDK for Python 提交训练作业时，它会创建一个[训练管道](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline)，该管道启动自定义作业在 Vertex AI 训练上进行训练。

In [ ]:
JOB_NAME = f"{APP_NAME}-pytorch-cstm-cntr-{get_timestamp()}"

print(f"APP_NAME={APP_NAME}")
print(f"CUSTOM_TRAIN_IMAGE_URI={CUSTOM_TRAIN_IMAGE_URI}")
print(f"JOB_NAME={JOB_NAME}")

In [ ]:
# configure the job with container image spec
job = aiplatform.CustomContainerTrainingJob(
    display_name=f"{JOB_NAME}", container_uri=f"{CUSTOM_TRAIN_IMAGE_URI}"
)

In [ ]:
# define training code arguments
training_args = ["--num-epochs", "2", "--model-name", "finetuned-bert-classifier"]

In [ ]:
# submit the custom job to Vertex AI training service
model = job.run(
    replica_count=1,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1,
    args=training_args,
    sync=False,
)

监控自定义作业的进度

您可以通过以下链接监控从云控制台启动的自定义作业[这里](https://console.cloud.google.com/vertex-ai/training/training-pipelines/)或使用gcloud CLI命令[`gcloud beta ai custom-jobs stream-logs`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs)

![在 Vertex AI 训练中监控自定义作业进度](./images/vertex-training-monitor-custom-job-container.png)

##### ***【可选】*** **使用gcloud CLI提交自定义容器的自定义作业**
您可以使用 [`gcloud beta ai custom-jobs create`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/create) 命令和自定义容器规范将训练作业提交给Vertex AI训练服务。`gcloud`命令提交训练作业并启动具有指定自定义容器镜像的工作池。

```
gcloud beta ai custom-jobs create \
    --display-name=${JOB_NAME} \
    --region ${REGION} \
    --worker-pool-spec=replica-count=1,machine-type='n1-standard-8',accelerator-type='NVIDIA_TESLA_V100',accelerator-count=1,container-image-uri=${CUSTOM_TRAIN_IMAGE_URI} \
    --args="--model-name","finetuned-bert-classifier","--job-dir",$JOB_DIR
```

- `worker-pool-spec` 参数定义了自定义作业使用的工作池配置。以下是`worker-pool-spec`中的字段：
    - 将`container-image-uri`设置为用于训练的推送到Google Cloud容器注册表的自定义容器镜像
    - 设置`accelerator-type`和`machine-type`以设置执行应用程序的计算类型

请参阅[文档](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/create#--args)获取更多详细信息。

位于`./custom_container/scripts/train-cloud.sh`的脚本包含了启动自定义作业和监视日志的`gcloud`命令。

In [ ]:
!cd custom_container && ./scripts/train-cloud.sh

除了云控制台之外，您还可以通过使用[gcloud CLI](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs)通过传递作业ID来通过流式日志监控作业进度：

```
gcloud ai custom-jobs stream-logs <job_id> --region=$REGION
```

您可以通过运行以下命令来验证写入GCS的模型文件：

```
!gsutil ls -l $JOB_DIR/
```

超参数调整

用于微调变压器模型以进行情感分析任务的训练应用程序代码使用超参数，例如学习率和权重衰减。这些超参数控制训练算法的行为，并可以对生成的模型的性能产生显著影响。笔记本的这一部分展示了如何使用Vertex AI训练服务自动调整这些超参数。

我们通过将训练应用程序代码和依赖项打包到Docker容器中，并将容器推送到Google容器注册表，类似于在Vertex AI上运行自定义容器进行Custom Job，向Vertex AI Training服务提交[超参数调整作业](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)。

![在Vertex AI Training上使用自定义容器进行超参数调整](./images/hp-tuning-with-custom-containers-on-vertex-training.png)

在Vertex AI中，超参数调整是如何工作的呢？

以下是在Vertex AI培训服务上运行超参数调整作业涉及的高级步骤：

- 您定义要调整模型的超参数，以及要优化的度量（或目标）
- Vertex AI使用您指定的超参数和限制运行多个训练应用程序的试验-要运行的最大试验次数和并行试验次数。 
- Vertex AI跟踪每个试验的结果，并为后续试验进行调整。这需要您的训练应用程序使用Python包[`cloudml-hypertune`](https://github.com/GoogleCloudPlatform/cloudml-hypertune)向Vertex AI报告指标。 
- 作业完成后，根据您配置的标准获取所有试验的摘要，该摘要基于最有效的值配置。

请参考[Vertex AI文档](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)了解如何配置和选择用于调整的超参数，配置调整策略以及Vertex AI如何优化超参数调整作业。默认的调整策略使用先前试验的结果来指导后续试验中值的分配。

### 用于超参数调整的训练应用代码更改

在Vertex AI中进行超参数调整有一些特定要求：

1. 要将超参数值传递给训练代码，您需要在主训练模块中为每个调整的超参数定义一个命令行参数。使用这些参数中传递的值来设置训练应用程序代码中相应的超参数。
1. 您必须将训练应用程序的指标传递给Vertex AI以评估试验的有效性。您可以使用`cloudml-hypertune` Python包来报告指标。

先前，在培训应用程序代码中，我们对transformer模型进行微调以进行情感分析任务，使用超参数作为训练参数（`training_args`）实例化[`Trainer`](https://huggingface.co/transformers/main_classes/trainer.html)。

这些超参数作为命令行参数传递给培训模块`trainer.task`，然后传递给`training_args`。请参考`./python_package/trainer`模块查看培训应用程序代码。

启用超参数调整时，为了向Vertex AI报告度量标准，我们在评估阶段之后调用 [`cloudml-hypertune` Python 包](https://github.com/GoogleCloudPlatform/cloudml-hypertune)。此包被添加为 `trainer` 的 [callback](https://huggingface.co/transformers/main_classes/callback.html#transformers.trainer_callback.TrainerCallback)。`trainer` 对象将由上一次评估阶段计算的度量传递给callback，以便由`hypertune`库报告给Vertex AI用于评估试验。

```python
# 添加用于报告度量的超参数调整回调函数
if args.hp_tune == "y":
    trainer.add_callback(HPTuneCallback("accuracy", "eval_accuracy"))

class HPTuneCallback(TrainerCallback):
    """
    一个自定义的回调类，用于在每个周期末向超调器报告度量标准。
    """
    
    def __init__(self, metric_tag, metric_value):
        super(HPTuneCallback, self).__init__()
        self.metric_tag = metric_tag
        self.metric_value = metric_value
        self.hpt = hypertune.HyperTune()
        
    def on_evaluate(self, args, state, control, **kwargs):
        print(f"HP metric {self.metric_tag}={kwargs['metrics'][self.metric_value]}")
        self.hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag=self.metric_tag,
            metric_value=kwargs['metrics'][self.metric_value],
            global_step=state.epoch)
```

在Vertex AI上运行超参数调整作业

在将超参数调整作业提交给Vertex AI之前，将带有训练应用程序的自定义容器映像推送到Google Cloud容器注册表，然后提交作业给Vertex AI。我们将使用用于在Vertex AI训练服务上运行自定义作业的相同映像。

验证容器注册表中的自定义容器镜像

In [ ]:
!gcloud container images describe $CUSTOM_TRAIN_IMAGE_URI

初始化Python的Vertex AI SDK

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

配置并提交超参数调整作业到Vertex AI培训服务

使用具有训练代码和其他依赖项的自定义容器映像配置[超参数调整作业](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning)，具体请参考[此处](https://cloud.google.com/vertex-ai/docs/training/create-custom-container)。

在配置和提交超参数调整作业时，您需要附加一个自定义作业定义，其中包括定义机器类型、加速器和表示自定义容器的容器映像的URI的工作池规范。

In [ ]:
JOB_NAME = f"{APP_NAME}-pytorch-hptune-{get_timestamp()}"

print(f"APP_NAME={APP_NAME}")
print(f"CUSTOM_TRAIN_IMAGE_URI={CUSTOM_TRAIN_IMAGE_URI}")
print(f"JOB_NAME={JOB_NAME}")

使用`hp-tune`参数设置为`y`来定义训练参数，以便训练应用程序代码可以向Vertex AI报告指标。

In [ ]:
training_args = [
    "--num-epochs",
    "2",
    "--model-name",
    "finetuned-bert-classifier",
    "--hp-tune",
    "y",
]

 使用工作池规范创建一个 CustomJob，以定义机器类型、加速器以及包含训练应用程序代码的客户端容器规范。

In [ ]:
# The spec of the worker pools including machine type and Docker image
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-8",
            "accelerator_type": "NVIDIA_TESLA_V100",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {"image_uri": CUSTOM_TRAIN_IMAGE_URI, "args": training_args},
    }
]

In [ ]:
custom_job = aiplatform.CustomJob(
    display_name=JOB_NAME, worker_pool_specs=worker_pool_specs
)

将`parameter_spec`定义为一个Python字典对象，其中包含搜索空间，即要搜索和优化的参数。键是作为命令行参数传递给训练代码的超参数名称，值是参数规范。规范要求将超参数数据类型指定为参数值规范的实例。

请参考[文档](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview#hyperparameters)有关选择要调整的超参数以及如何定义参数规范的信息。

In [ ]:
# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
parameter_spec = {
    "learning-rate": hpt.DoubleParameterSpec(min=1e-6, max=0.001, scale="log"),
    "weight-decay": hpt.DiscreteParameterSpec(
        values=[0.0001, 0.001, 0.01, 0.1], scale=None
    ),
}

定义`metric_spec`，其中包含指标的名称和目标以优化指标。目标指定您是否希望调整模型以最大化或最小化此指标的值。

In [ ]:
# Dictionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec = {"accuracy": "maximize"}

使用自定义作业、指标规范、参数规范和试验限制配置并提交超参数调整作业。

- **`max_trial_count`**：服务运行的最大试验次数。我们建议从较小的值开始，以便了解所选择的超参数对结果的影响，然后再逐渐增加。
- **`parallel_trial_count`**：并行运行的试验数量。我们建议从较小的值开始，因为Vertex AI会使用先前试验的结果来指导后续试验中数值的分配。大量并行试验意味着这些试验将在没有进行中的任何试验结果的情况下开始。
- **`search_algorithm`**：为研究指定的搜索算法。如果您没有指定算法，Vertex AI 默认应用贝叶斯优化来找到在参数空间中搜索最优解。

请参考[文档](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning#configuration)了解超参数训练作业的配置。

In [ ]:
hp_job = aiplatform.HyperparameterTuningJob(
    display_name=JOB_NAME,
    custom_job=custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=5,
    parallel_trial_count=2,
    search_algorithm=None,
)

In [ ]:
model = hp_job.run(sync=False)

监视自定义作业的进度

您可以通过以下链接监视从Cloud Console启动的超参数调整作业[这里](https://console.cloud.google.com/vertex-ai/training/hyperparameter-tuning-jobs/)，或者使用gcloud CLI命令 [`gcloud beta ai custom-jobs stream-logs`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs)

![在Vertex AI Training中监视超参数调整作业的进展](./images/vertex-training-monitor-hptuning-job-container.png)

工作完成后，您可以查看和格式化超参数调整试验（由Vertex AI训练服务运行）的结果作为Pandas数据帧。

In [ ]:
def get_trials_as_df(trials):
    results = []
    for trial in trials:
        row = {}
        t = MessageToDict(trial._pb)
        # print(t)
        row["Trial ID"], row["Status"], row["Start time"], row["End time"] = (
            t["id"],
            t["state"],
            t["startTime"],
            t.get("endTime", None),
        )

        for param in t["parameters"]:
            row[param["parameterId"]] = param["value"]

        if t["state"] == "SUCCEEDED":
            row["Training step"] = t["finalMeasurement"]["stepCount"]
            for metric in t["finalMeasurement"]["metrics"]:
                row[metric["metricId"]] = metric["value"]
        results.append(row)

    _df = pd.DataFrame(results)
    return _df

In [ ]:
df_trials = get_trials_as_df(hp_job.trials)
df_trials

现在，您可以从试验结果中选择表现最好的试验部署到 Vertex AI 预测。

In [ ]:
# get trial id of the best run from the Trials
best_trial_id = df_trials.loc[df_trials["accuracy"].idxmax()]["Trial ID"]

In [ ]:
# get base output directory where artifacts are saved
base_output_dir = MessageToDict(hp_job._gca_resource._pb)["trialJobSpec"][
    "baseOutputDirectory"
]["outputUriPrefix"]

# get the model artifacts of the best trial id
best_model_artifact_uri = f"{base_output_dir}/{best_trial_id}"

print(
    f"Model artifacts from the Hyperparameter Tuning Job with bbest trial id {best_trial_id} are located at {best_model_artifact_uri}"
)

您可以通过运行以下命令来验证写入GCS的模型文件：

In [ ]:
!gsutil ls -r $best_model_artifact_uri/

##### **[可选]** 使用 gcloud CLI 提交超参数调整作业
您可以使用[`gcloud beta ai hp-tuning-jobs create`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/hp-tuning-jobs/create)命令将超参数调整作业提交给Vertex AI训练服务。`gcloud`命令将提交超参数调整作业并根据自定义容器映像、试验数量和设置的标准启动多个试验工作池。该命令需要将超参数调整作业配置提供为YAML格式的配置文件，包含作业名称。

以下是如何使用`gcloud`CLI在Vertex AI上提交超参数调整作业的示例：

In [ ]:
%%bash -s $BUCKET_NAME $APP_NAME

# ========================================================
# set job parameters
# ========================================================
# PROJECT_ID: Change to your project id
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
        
# set job display name
JOB_PREFIX="finetuned-bert-classifier"
JOB_NAME=${JOB_PREFIX}-pytorch-hptune-$(date +%Y%m%d%H%M%S)
echo "Launching hyperparameter tuning job with display name as "$JOB_NAME

# BUCKET_NAME is a required parameter to run the cell.
BUCKET_NAME=$1

# APP_NAME: get application name
APP_NAME=$2

# JOB_DIR: Where to store prepared package and upload output model.
JOB_DIR=${BUCKET_NAME}/${JOB_PREFIX}/model/${JOB_NAME}

# custom container image URI
CUSTOM_TRAIN_IMAGE_URI='gcr.io/'${PROJECT_ID}'/pytorch_gpu_train_'${APP_NAME}

# ========================================================
# create hyperparameter tuning configuration file
# ========================================================
cat << EOF > ./python_package/hptuning_job.yaml

studySpec:
  metrics:
  - metricId: accuracy
    goal: MAXIMIZE
  parameters:
  - parameterId: learning-rate
    scaleType: UNIT_LOG_SCALE
    doubleValueSpec:
      minValue: 0.000001
      maxValue: 0.001
  - parameterId: weight-decay
    scaleType: SCALE_TYPE_UNSPECIFIED
    discreteValueSpec:
      values: [
          0.0001, 
          0.001, 
          0.01, 
          0.1
      ]
  measurementSelectionType: BEST_MEASUREMENT
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: n1-standard-8
      acceleratorType: NVIDIA_TESLA_V100
      acceleratorCount: 1  
    replicaCount: 1
    containerSpec:
      imageUri: $CUSTOM_TRAIN_IMAGE_URI
      args: ["--num-epochs", "2", "--model-name", "finetuned-bert-classifier", "--hp-tune", "y"]
  baseOutputDirectory: 
    outputUriPrefix: $JOB_DIR/
EOF

# ========================================================
# submit hyperparameter tuning job
# ========================================================
gcloud beta ai hp-tuning-jobs create \
   --config ./python_package/hptuning_job.yaml \
   --display-name $JOB_NAME \
   --algorithm algorithm-unspecified \
   --max-trial-count 5 \
   --parallel-trial-count 2 \
   --region=us-central1

部署

在[Vertex AI Predictions](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)上部署PyTorch模型需要使用一个自定义容器来提供在线预测。您将部署一个运行[PyTorch的TorchServe](https://pytorch.org/serve/)工具的容器，以便从Hugging Face Transformers Fine-Tuned Transformer模型对情感分析任务进行预测。然后您可以使用Vertex AI Predictions来对输入文本的情感进行分类。

### 使用自定义容器在Vertex AI Predictions上部署模型

要使用自定义容器提供PyTorch模型的预测，您必须提供一个Docker容器映像给Vertex AI，该容器可以运行一个HTTP服务器，例如这种情况下的TorchServe。请参考[文档](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements)，其中描述了与Vertex AI Predictions兼容的容器映像的要求。

![在Vertex AI Predictions上使用自定义容器提供PyTorch模型的预测](./images/serve-pytorch-model-on-vertex-predictions-with-custom-containers.png)

基本上，要在Vertex AI Predictions上部署PyTorch模型，以下是步骤：

1. 打包经过训练的模型工件，包括[默认](https://pytorch.org/serve/#default-handlers)或[自定义](https://pytorch.org/serve/custom_service.html)处理程序，通过使用[Torch模型存档](https://github.com/pytorch/serve/tree/master/model-archiver)创建一个存档文件
2. 构建一个[兼容Vertex AI Predictions](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements)的自定义容器，以使用Torchserve提供模型服务
3. 将带有自定义容器映像的模型上传为Vertex AI模型资源以提供预测
4. 创建一个Vertex AI端点和[部署模型](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)资源

创建一个自定义模型处理程序来处理预测请求

当使用经过微调的变压器模型来预测输入文本的情绪时，需要对输入文本进行预处理，并通过添加名称（积极/消极）到目标标签（1/0）以及概率（或置信度）进行后处理。我们创建一个自定义处理程序脚本，该脚本与模型工件打包在一起，当TorchServe运行时执行该代码。

自定义处理程序脚本执行以下操作：

- 在将输入文本发送到模型以进行推理之前对输入文本进行预处理
- 自定义调用模型进行推理的方式
- 在将模型的输出后处理后发送回响应

请参考[TorchServe文档](https://pytorch.org/serve/custom_service.html)来定义自定义处理程序。

In [ ]:
%%writefile predictor/custom_handler.py

import os
import json
import logging

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from ts.torch_handler.base_handler import BaseHandler

logger = logging.getLogger(__name__)


class TransformersClassifierHandler(BaseHandler):
    """
    The handler takes an input string and returns the classification text 
    based on the serialized transformers checkpoint.
    """
    def __init__(self):
        super(TransformersClassifierHandler, self).__init__()
        self.initialized = False

    def initialize(self, ctx):
        """ Loads the model.pt file and initialized the model object.
        Instantiates Tokenizer for preprocessor to use
        Loads labels to name mapping file for post-processing inference response
        """
        self.manifest = ctx.manifest

        properties = ctx.system_properties
        model_dir = properties.get("model_dir")
        self.device = torch.device("cuda:" + str(properties.get("gpu_id")) if torch.cuda.is_available() else "cpu")

        # Read model serialize/pt file
        serialized_file = self.manifest["model"]["serializedFile"]
        model_pt_path = os.path.join(model_dir, serialized_file)
        if not os.path.isfile(model_pt_path):
            raise RuntimeError("Missing the model.pt or pytorch_model.bin file")
        
        # Load model
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        self.model.to(self.device)
        self.model.eval()
        logger.debug('Transformer model from path {0} loaded successfully'.format(model_dir))
        
        # Ensure to use the same tokenizer used during training
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

        # Read the mapping file, index to object name
        mapping_file_path = os.path.join(model_dir, "index_to_name.json")

        if os.path.isfile(mapping_file_path):
            with open(mapping_file_path) as f:
                self.mapping = json.load(f)
        else:
            logger.warning('Missing the index_to_name.json file. Inference output will default.')
            self.mapping = {"0": "Negative",  "1": "Positive"}

        self.initialized = True

    def preprocess(self, data):
        """ Preprocessing input request by tokenizing
            Extend with your own preprocessing steps as needed
        """
        text = data[0].get("data")
        if text is None:
            text = data[0].get("body")
        sentences = text.decode('utf-8')
        logger.info("Received text: '%s'", sentences)

        # Tokenize the texts
        tokenizer_args = ((sentences,))
        inputs = self.tokenizer(*tokenizer_args,
                                padding='max_length',
                                max_length=128,
                                truncation=True,
                                return_tensors = "pt")
        return inputs

    def inference(self, inputs):
        """ Predict the class of a text using a trained transformer model.
        """
        prediction = self.model(inputs['input_ids'].to(self.device))[0].argmax().item()

        if self.mapping:
            prediction = self.mapping[str(prediction)]

        logger.info("Model predicted: '%s'", prediction)
        return [prediction]

    def postprocess(self, inference_output):
        return inference_output


生成目标标签以命名文件 *[可选]*

在自定义处理程序中，我们引用了一个目标标签与其有意义名称之间的映射文件，该文件将用于格式化预测响应。在这里，我们将目标标签"0"映射为"Negative"，将"1"映射为"Positive"。

In [ ]:
%%writefile ./predictor/index_to_name.json

{
    "0": "Negative", 
    "1": "Positive"
}

#### **创建自定义容器映像以提供预测**

我们将使用Cloud Build来创建带有以下构建步骤的自定义容器映像：

##### **下载模型工件**

从Cloud Storage下载作为训练（或超参数调整）任务的一部分保存的模型工件到本地目录。

In [ ]:
GCS_MODEL_ARTIFACTS_URI = best_model_artifact_uri

在云存储桶中验证模型文件artifact。

In [ ]:
!gsutil ls -r $GCS_MODEL_ARTIFACTS_URI/model/

从云存储复制文件到本地目录

In [ ]:
!gsutil -m cp -r $GCS_MODEL_ARTIFACTS_URI/model/ ./predictor/

In [ ]:
!ls -ltrR ./predictor/model

构建容器镜像

创建一个以TorchServe为基础镜像的Dockerfile：

- **`RUN`**：安装依赖项，如`transformers`
- **`COPY`**：将模型文件添加到容器镜像的`/home/model-server/`目录中
- **`COPY`**：将自定义处理程序脚本添加到容器镜像的`/home/model-server/`目录中
- **`RUN`**：创建`/home/model-server/config.properties`来定义服务配置（健康和预测监听端口）
- **`RUN`**：运行[Torch模型打包器](https://pytorch.org/serve/model-archiver.html)来从复制到镜像`/home/model-server/`中的文件创建一个模型存档文件。模型存档文件保存在`/home/model-server/model-store/`中，名称与`<model-name>.mar`相同
- **`CMD`**：启动Torchserve HTTP服务器，引用配置属性并启用该模型的服务

In [ ]:
%%bash -s $APP_NAME

APP_NAME=$1

cat << EOF > ./predictor/Dockerfile

FROM pytorch/torchserve:latest-cpu

# install dependencies
RUN python3 -m pip install --upgrade pip
RUN pip3 install transformers

USER model-server

# copy model artifacts, custom handler and other dependencies
COPY ./custom_handler.py /home/model-server/
COPY ./index_to_name.json /home/model-server/
COPY ./model/$APP_NAME/ /home/model-server/

# create torchserve configuration file
USER root
RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
RUN printf "\ninference_address=http://0.0.0.0:7080" >> /home/model-server/config.properties
RUN printf "\nmanagement_address=http://0.0.0.0:7081" >> /home/model-server/config.properties
USER model-server

# expose health and prediction listener ports from the image
EXPOSE 7080
EXPOSE 7081

# create model archive file packaging model artifacts and dependencies
RUN torch-model-archiver -f \
  --model-name=$APP_NAME \
  --version=1.0 \
  --serialized-file=/home/model-server/pytorch_model.bin \
  --handler=/home/model-server/custom_handler.py \
  --extra-files "/home/model-server/config.json,/home/model-server/tokenizer.json,/home/model-server/training_args.bin,/home/model-server/tokenizer_config.json,/home/model-server/special_tokens_map.json,/home/model-server/vocab.txt,/home/model-server/index_to_name.json" \
  --export-path=/home/model-server/model-store

# run Torchserve HTTP serve to respond to prediction requests
CMD ["torchserve", \
     "--start", \
     "--ts-config=/home/model-server/config.properties", \
     "--models", \
     "$APP_NAME=$APP_NAME.mar", \
     "--model-store", \
     "/home/model-server/model-store"]
EOF

echo "Writing ./predictor/Dockerfile"

构建带有容器注册表（gcr.io）路径标记的Docker镜像

In [ ]:
CUSTOM_PREDICTOR_IMAGE_URI = f"gcr.io/{PROJECT_ID}/pytorch_predict_{APP_NAME}"
print(f"CUSTOM_PREDICTOR_IMAGE_URI = {CUSTOM_PREDICTOR_IMAGE_URI}")

In [ ]:
!docker build \
  --tag=$CUSTOM_PREDICTOR_IMAGE_URI \
  ./predictor

在将容器映像推送到容器注册表以在 Vertex AI 预测中使用之前，您可以在本地环境中将其作为容器运行，以验证服务器是否按预期工作。【可选】

运行容器图像作为本地容器，运行以下命令：

In [ ]:
!docker stop local_bert_classifier
!docker run -t -d --rm -p 7080:7080 --name=local_bert_classifier $CUSTOM_PREDICTOR_IMAGE_URI
!sleep 20

发送容器的服务器健康检查，请运行以下命令：

In [ ]:
!curl http://localhost:7080/ping

如果成功，服务器会返回以下响应：

```
{
  "status": "Healthy"
}
```

3. 要向容器的服务器发送预测请求，请运行以下命令：

In [ ]:
%%bash -s $APP_NAME

APP_NAME=$1

cat > ./predictor/instances.json <<END
{ 
   "instances": [
     { 
       "data": {
         "b64": "$(echo 'Take away the CGI and the A-list cast and you end up with film with less punch.' | base64 --wrap=0)"
       }
     }
   ]
}
END

curl -s -X POST \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @./predictor/instances.json \
  http://localhost:7080/predictions/$APP_NAME/

这个请求使用一个测试句子。如果成功，服务器以以下格式返回预测：

```
    {"predictions": ["Negative"]}
```

4. 要停止容器，请运行以下命令：

In [ ]:
!docker stop local_bert_classifier

部署服务容器到Vertex AI预测

我们在Vertex AI上创建一个模型资源并将模型部署到Vertex AI端点。在使用模型之前，您必须将模型部署到一个端点。部署的模型会运行自定义容器镜像来提供预测。

将serving容器推送到容器注册表中

将包含推断代码和依赖项的容器镜像推送到您的容器注册表

In [ ]:
!docker push $CUSTOM_PREDICTOR_IMAGE_URI

初始化Python的Vertex AI SDK

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

使用自定义serving容器创建模型资源

In [ ]:
VERSION = 1
model_display_name = f"{APP_NAME}-v{VERSION}"
model_description = "PyTorch based text classifier with custom container"

MODEL_NAME = APP_NAME
health_route = "/ping"
predict_route = f"/predictions/{MODEL_NAME}"
serving_container_ports = [7080]

In [ ]:
model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=CUSTOM_PREDICTOR_IMAGE_URI,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

model.wait()

print(model.display_name)
print(model.resource_name)

有关上传或导入模型的更多上下文，请参考[文档](https://cloud.google.com/vertex-ai/docs/general/import-model)。

为具有自定义容器的模型创建一个端点

In [ ]:
endpoint_display_name = f"{APP_NAME}-endpoint"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)

部署模型到端点

部署模型会将物理资源与模型关联起来，使其能够以低延迟提供在线预测。

**注意：** 这一步需要几分钟时间来部署资源。

In [ ]:
traffic_percentage = 100
machine_type = "n1-standard-4"
deployed_model_display_name = model_display_name
min_replica_count = 1
max_replica_count = 3
sync = True

model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=deployed_model_display_name,
    machine_type=machine_type,
    traffic_percentage=traffic_percentage,
    sync=sync,
)

使用Vertex AI SDK调用部署的模型的端点进行预测

获取端点ID

In [ ]:
endpoint_display_name = f"{APP_NAME}-endpoint"
filter = f'display_name="{endpoint_display_name}"'

for endpoint_info in aiplatform.Endpoint.list(filter=filter):
    print(
        f"Endpoint display name = {endpoint_info.display_name} resource id ={endpoint_info.resource_name} "
    )

endpoint = aiplatform.Endpoint(endpoint_info.resource_name)

In [ ]:
endpoint.list_models()

##### **为在线预测格式化输入**

此笔记本使用[Torchserve的KServe基于推理API](https://pytorch.org/serve/inference_api.html#kserve-inference-api)，该API也是[Vertex AI Predictions兼容格式](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements#prediction)。 对于在线预测请求，请按照以下方式将预测输入实例格式化为带有base64编码的JSON：

```
[
    {
        "data": {
            "b64": "<base64编码字符串>"
        }
    }
]
```

定义样本文本以测试预测

In [ ]:
test_instances = [
    b"Jaw dropping visual affects and action! One of the best I have seen to date.",
    b"Take away the CGI and the A-list cast and you end up with film with less punch.",
]

发送在线预测请求

格式化输入文本字符串，使用格式化的输入请求调用预测终端点，并获取响应。

In [ ]:
print("=" * 100)
for instance in test_instances:
    print(f"Input text: \n\t{instance.decode('utf-8')}\n")
    b64_encoded = base64.b64encode(instance)
    test_instance = [{"data": {"b64": f"{str(b64_encoded.decode('utf-8'))}"}}]
    print(f"Formatted input: \n{json.dumps(test_instance, indent=4)}\n")
    prediction = endpoint.predict(instances=test_instance)
    print(f"Prediction response: \n\t{prediction}")
    print("=" * 100)

**[可选]** **使用gcloud CLI发出预测请求**

您也可以调用Vertex AI终端节点来使用[`gcloud beta ai endpoints predict`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/endpoints/predict)进行预测。

以下代码展示了如何使用`gcloud` CLI向Vertex AI终端节点发出预测请求：

In [ ]:
endpoint_display_name = f"{APP_NAME}-endpoint"

In [ ]:
%%bash -s $REGION $endpoint_display_name

REGION=$1
endpoint_display_name=$2

# get endpoint id
echo "REGION = ${REGION}"
echo "ENDPOINT DISPLAY NAME = ${endpoint_display_name}"
endpoint_id=$(gcloud beta ai endpoints list --region ${REGION} --filter "display_name=${endpoint_display_name}" --format "value(ENDPOINT_ID)")
echo "ENDPOINT_ID = ${endpoint_id}"

# call prediction endpoint
input_text="Take away the CGI and the A-list cast and you end up with film with less punch."
echo "INPUT TEXT = ${input_text}"

prediction=$(
echo """
{ 
   "instances": [
     { 
       "data": {
         "b64": "$(echo ${input_text} | base64 --wrap=0)"
       }
     }
   ]
}
""" | gcloud beta ai endpoints predict ${endpoint_id} --region=$REGION --json-request -)

echo "PREDICTION RESPONSE = ${prediction}"

整理清理

### 整理清理训练和部署资源

要清理本笔记中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 训练作业
- 模型
- 端点
- 云存储桶
- 容器镜像

为要删除的资源类型设置标志。

In [ ]:
delete_custom_job = False
delete_hp_tuning_job = False
delete_endpoint = True
delete_model = False
delete_bucket = False
delete_image = False

定义作业、模型和端点的客户端

In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = f"projects/{PROJECT_ID}/locations/{REGION}"

client_options = {"api_endpoint": API_ENDPOINT}

# Initialize Vertex AI SDK
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

In [ ]:
# functions to create client
def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()

在笔记本中定义函数来列出以`APP_NAME`开头的工作、模型和端点。

In [ ]:
def list_custom_jobs():
    client = clients["job"]
    jobs = []
    response = client.list_custom_jobs(parent=PARENT)
    for row in response:
        _row = MessageToDict(row._pb)
        if _row["displayName"].startswith(APP_NAME):
            jobs.append((_row["name"], _row["displayName"]))
    return jobs


def list_hp_tuning_jobs():
    client = clients["job"]
    jobs = []
    response = client.list_hyperparameter_tuning_jobs(parent=PARENT)
    for row in response:
        _row = MessageToDict(row._pb)
        if _row["displayName"].startswith(APP_NAME):
            jobs.append((_row["name"], _row["displayName"]))
    return jobs


def list_models():
    client = clients["model"]
    models = []
    response = client.list_models(parent=PARENT)
    for row in response:
        _row = MessageToDict(row._pb)
        if _row["displayName"].startswith(APP_NAME):
            models.append((_row["name"], _row["displayName"]))
    return models


def list_endpoints():
    client = clients["endpoint"]
    endpoints = []
    response = client.list_endpoints(parent=PARENT)
    for row in response:
        _row = MessageToDict(row._pb)
        if _row["displayName"].startswith(APP_NAME):
            print(_row)
            endpoints.append((_row["name"], _row["displayName"]))
    return endpoints

删除自定义训练作业

In [ ]:
# Delete the custom training using the Vertex AI fully qualified identifier for the custom training
try:
    if delete_custom_job:
        custom_jobs = list_custom_jobs()
        for job_id, job_name in custom_jobs:
            print(f"Deleting job {job_id} [{job_name}]")
            clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

删除超参数调整作业

In [ ]:
# Delete the hyperparameter tuning jobs using the Vertex AI fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hp_tuning_job:
        hp_tuning_jobs = list_hp_tuning_jobs()
        for job_id, job_name in hp_tuning_jobs:
            print(f"Deleting job {job_id} [{job_name}]")
            clients["job"].delete_hyperparameter_tuning_job(name=job_id)
except Exception as e:
    print(e)

取消部署模型和删除端点

In [ ]:
# Delete the endpoint using the Vertex AI fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        endpoints = list_endpoints()
        for endpoint_id, endpoint_name in endpoints:
            endpoint = aiplatform.Endpoint(endpoint_id)
            # undeploy models from the endpoint
            print(f"Undeploying all deployed models from the endpoint {endpoint_name}")
            endpoint.undeploy_all(sync=True)
            # deleting endpoint
            print(f"Deleting endpoint {endpoint_id} [{endpoint_name}]")
            clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

删除模型

In [ ]:
# Delete the model using the Vertex AI fully qualified identifier for the model
try:
    models = list_models()
    for model_id, model_name in models:
        print(f"Deleting model {model_id} [{model_name}]")
        clients["model"].delete_model(name=model_id)
except Exception as e:
    print(e)

删除暂存桶中的内容

---

***注意：此云存储桶中的所有内容将被删除。请谨慎运行。***

In [ ]:
if delete_bucket and "BUCKET_NAME" in globals():
    print(f"Deleting all contents from the bucket {BUCKET_NAME}")

    shell_output = ! gsutil du -as $BUCKET_NAME
    print(
        f"Size of the bucket {BUCKET_NAME} before deleting = {shell_output[0].split()[0]} bytes"
    )

    # uncomment below line to delete contents of the bucket
    # ! gsutil rm -r $BUCKET_NAME

    shell_output = ! gsutil du -as $BUCKET_NAME
    if float(shell_output[0].split()[0]) > 0:
        print(
            "PLEASE UNCOMMENT LINE TO DELETE BUCKET. CONTENT FROM THE BUCKET NOT DELETED"
        )

    print(
        f"Size of the bucket {BUCKET_NAME} after deleting = {shell_output[0].split()[0]} bytes"
    )

删除容器注册表中的图像

从注册表中删除本教程中由变量 `APP_NAME` 定义前缀所创建的所有容器图像。所有关联的标签也将被删除。

In [ ]:
gcr_images = !gcloud container images list --repository=gcr.io/$PROJECT_ID --filter="name~"$APP_NAME

if delete_image:
    for image in gcr_images:
        if image != "NAME":  # skip header line
            print(f"Deleting image {image} including all tags")
            !gcloud container images delete $image --force-delete-tags --quiet

### 清理笔记本环境

在实验完成后，您可以选择[停止](https://cloud.google.com/ai-platform/notebooks/docs/shut-down)或者删除 AI 笔记本实例，以避免产生任何费用。如果您想保存您的工作，可以选择停止实例。

```
# 停止笔记本实例
gcloud notebooks instances stop example-instance --location=us-central1-a


# 删除笔记本实例
gcloud notebooks instances delete example-instance --location=us-central1-a
```